# Setup

In [26]:
import numpy as np
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
from datetime import datetime
import random
import collections
import pandas as pd
import seaborn as sns
np.random.seed(42)
pd.set_option('display.max_columns', None)
%matplotlib inline
import numpy as np
from math import sqrt
import matplotlib.pyplot as plt
import statsmodels.api as sm

plt.rcParams['axes.grid'] = True

# Strategy Function (Signal Generation)

In [3]:
def comp_strat(high, low, A, B, C, D, alpha, T, initial_kalman, initial_pred = "default"):
    # trading within each chunk
    '''This function returns, for a pair of securities over a given chunk,
            - sequence of estimated hidden spreads x_k
            - sequence of uncertainty around hidden spreads (x_k)  R_k
            - strategies chosen strat
            - the final values from the kalman filter final_kalman to be passed on
            - the final predictions for spread and unc. final_pred to be passed on
        given the inputs:
            - the price sequences high, low
            - the estimated parameters A, B, C, D
            - the uncertainty treshold alpha (equivalent to zscore)  (free parameter)
            - holding period for each trade T (pre-chosen)
            - initial values for the kalman filter initial_kalman
            - initial predictions predictions for spread and unc. initial_pred
        using the Kalman Filter.
    '''
    
    y = high - low
    start = 1 + T
    
    # compute best estimate of hidden spread
    x, R, sigma, K = kalman_filter(y, A = A, B = B, C = C, D = D, initial =  initial_kalman)
    x = np.asarray(x)
    R = np.asarray(R)
    
    # initialise conditional expectation on past observation
    x_k = np.zeros(len(x))
    R_k = np.zeros(len(x))
    strat = np.zeros(len(x))
    
    # Adjusting holding period based on half life (if half life is within the specified range)
    T1 = Half_Life()
    T1.apply_half_life(x)
    if (T1.use()):
        T = T1.half_life
    
    # compute strategy for initial period
    if initial_pred != "default": 
        x_k[0] = initial_pred[0]
        R_k[0] =  initial_pred[1]
        if (y[0] - x_k[0]) / np.sqrt(R_k[0]) > alpha:
            strat[0] = 1
        elif (y[0] - x_k[0]) / np.sqrt(R_k[0]) < - alpha:
            strat[0] = -1
        else:
            strat[0] = 0
        
    # compute strategy for each period where 1 = short "high" and long "low" (long spread),
    # -1 vice versa (short spread), and 0 = do not trade  
    for i in range(1, len(x)):
        x_k[i] = A + B * x[i - 1]
        R_k[i] = (B ** 2) * R[i - 1] + C ** 2
        if (y[i] - x_k[i]) / np.sqrt(R_k[i]) > alpha:
            strat[i] = 1
        elif (y[i] - x_k[i]) / np.sqrt(R_k[i]) < - alpha:
            strat[i] = -1
        else:
            strat[i] = 0
            
    # compute ratio of low stock to high stock ???
    r = low / high
        
    # compute final kalman
    final_kalman = (x[-1], R[-1])
    
    # compute x_N+1 | N and R_N+1 | N to pass on to next chunk
    x_N_plus_1 = A + B * x[-1] 
    R_N_plus_1 = B ** 2 * R[-1] + C ** 2
    final_pred = (x_N_plus_1, R_N_plus_1)
    
    return x_k, R_k, strat, final_kalman, final_pred

In [4]:
def pairs_trading(high, low, alpha, 
                  T, k_init = 100, 
                  k = 50, p = "all", 
                  max_iterations = 1000, 
                  threshold = 10e-10, 
                  initial_eta = (1.2, 0.5, 0.3, 0.7),
                  initial_kalman = (0, 0.1)):
    ''' This function returns, for a pair of securities,
            - the profit following the strategies profit
            - the cumulative profit following the strategies cum_profit
            - sequence of estimated hidden spreads x
            - sequence of uncertainty around hidden spreads R
            - strategies chosen strat 
            - sequence of observed spreads y
        evaluated on all chunks but the first, given the inputs
            - the price sequences high, low
            - the uncertainty treshold alpha (free parameter)
            - holding period for each trade T (free parameter)
            - the size of the first training chunk k_init
            - the size of the following chunks k 
            - the number of previous chunks to be considered 
              when computing the strategy p (takes integer values or "all")
            - max_iterations for the EM convergence 
            - threshold for EM conergence
            - initial parameters initial_eta 
            - intial values for the kalman filter intial_kalman
        using the Kalman Filter and the EM-smoothing-algorithm. 
    '''
    # scale
    high = 100 * high / low[0] 
    low = 100 * low / low[0]
    
    y = high - low
    n = len(y)

    # split data into chunks 
    high_chunks = np.array_split(high[k_init:], int((n - k_init) / k))
    low_chunks = np.array_split(low[k_init:], int((n - k_init) / k))
    y_chunks = np.array_split(y[k_init:], int((n - k_init) / k))
    
    high_chunks.insert(0, high[:k_init])
    low_chunks.insert(0, low[:k_init])
    y_chunks.insert(0, y[:k_init])
    
    # run EM-algorithm on initial chunk
    A, B, C, D, eta, x_N = EM(y_chunks[0],
                                 max_iterations = max_iterations, 
                                 threshold = threshold,
                                 initial_eta = initial_eta,
                                 initial_kalman = initial_kalman)
    
    # check that B is leq 1
    if B > 1:
        raise('ERROR: B>1')
        
    # compute strategy on the second chunk and initialise output 
    x, R, strat, final_kalman, final_pred = comp_strat(high = high_chunks[1],
                                             low = low_chunks[1],
                                             A = A,
                                             B = B,
                                             C = C,
                                             D = D,
                                             alpha = alpha,
                                             T = T, 
                                             initial_kalman = "default", 
                                             initial_pred = "default")
    
    # reestimate parameters and compute profit chunkwise         
    for i in range(1, int((n - k_init) / k)):
        if p == "all": #Using all chunks
            A, B, C, D, eta, x_N = EM(y[:(k_init + i * k)],
                                         max_iterations = max_iterations, 
                                         threshold = threshold,
                                         initial_eta = (A, B, C, D),
                                         initial_kalman = initial_kalman)
        else: #Using only certain number of chunks
            if i - p < 0: 
                beg = 0 
            else: 
                beg =  i - p + 1  
                
            A, B, C, D, eta, x_N = EM(np.concatenate(y_chunks[beg:(i + 1)]), 
                                         max_iterations = max_iterations,
                                         threshold = threshold,
                                         initial_eta = (A, B, C, D),
                                         initial_kalman = initial_kalman)
        if B > 1:
            raise('ERROR: B>1')
            
        res = comp_strat(high = high_chunks[i + 1],
                         low = low_chunks[i + 1],
                         A = A,
                         B = B,
                         C = C,
                         D = D,
                         alpha = alpha,
                         T = T, 
                         initial_kalman = final_kalman, 
                         initial_pred = final_pred) 
        
        x = np.hstack((x, res[0]))
        R = np.hstack((R, res[1]))
        strat = np.hstack((strat, res[2]))
        final_kalman = res[3]
        final_pred = res[4]
    
    x = np.hstack((np.zeros(k_init), x))
    R = np.hstack((np.zeros(k_init), R))
    strat = np.hstack((np.zeros(k_init), strat))
    
    # ratio of low stock to high stock
    r = low/high
    
    # compute profit 
    profit = np.zeros(n)
    
    for i in range(T, n): 
        profit[i] = strat[i - T] * (r[i - T]  * (high[i - T]  -  high[i])  
                                    +  low[i] - low[i - T])
        
    cum_profit = np.cumsum(profit)

    return profit, cum_profit, x, R, strat, y

# Load data

In [ ]:
def backtest(data, rep=5, start_time=, alpha=, T=)
    '''Input 
    - data: dataframe of the return data
    - rep: numbers of trading period(before pairs changed, default is 5)'''
    ###Cointegration test & pairs selection
    
    
    
    
    ### tic_high: list of tickers for the one stock of the pairs (list of list)
    ### tic_low: list of tickers for the other stock of the pairs
    profit_total = []
    cum_total = []
    
    
    for i in range(rep):
        high_pairs = data.loc[ : , tic_high[i]]
        low_pairs = data.loc[ : , tic_low[i]]
        npairs = len(tic_high[i])
        profit_ck =  []
        cum_ck = []
        
        for j in range(npairs):
            high = high_pairs.iloc[ : , j]
            low = low_pairs.iloc[ : , j]
            
            ### pairs trading for each pair of stocks in one period
            profit, cum_profit, x, R, strat, y = pairs_trading(high, low, alpha=alpha, 
                  T=T, k_init = 100, 
                  k = 50, p = "all", 
                  max_iterations = 1000, 
                  threshold = 10e-10, 
                  initial_eta = (1.2, 0.5, 0.3, 0.7),
                  initial_kalman = (0, 0.1))
            
            profit_ck = np.hstack(profit_ck, profit)
            cum_ck = np.hstack(cum_ck, cum_profit)
            
        temp = profit_ck.mean(1)
        profit_total = np.concatenate(profit_total,temp)
        temp = cum_ck.mean(1)
        cum_total = np.concatenate(cum_total,temp)
        
    return profit_total, cum_total
            
            
            
            
    
    

# Kalman Filter and EM Algorithm Functions

In [6]:
def generate(x0, A, B, C, D, num_obs = 100):
    '''creates sequences of true state (x)
       and observations (y)'''
    x = [x0] + [None] * (num_obs - 1)
    y = [None] * (num_obs)

    for k in range(num_obs - 1):
        x[k+1] = A + B * x[k] + C * np.random.normal()
        y[k] = x[k] + D * np.random.normal()

    y[k+1] = x[k+1] + D * np.random.normal()
    return x, y

def kalman_filter(y, A, B, C, D, initial = 'default'):
    '''takes in a list of observations y and
        estimates for A, B, C, D as outlined in
        Elliot (2005) and returns estimate of 
        the true state using Kalman filters.
        
        initial is a tuple of the initial values of
        the estimate of the state and it's variance
        in period zero.'''
    
    if initial == 'default':
        initial = (y[0], D ** 2)
    
    N = len(y) - 1 
    x = [0] * (N + 1) # state estimations
    R = [0] * (N + 1) # variance estimations sigma k | k
    sigma = [None] * (N + 1) # sigma k + 1 | k
    
    #initializing x and R
    x[0], R[0] = initial

    for k in range(N):
        # Equation 22
        x_bar = A + B * x[k]

        # Equation 23
        sigma[k+1] = B ** 2 * R[k] + C ** 2

        # Equation 24
        K = sigma[k+1] / (sigma[k+1] + D ** 2)

        # Equation 25
        x[k+1] = x_bar + K * (y[k+1] - x_bar)

        # Equation 26
        R[k+1] = sigma[k+1] * (1 - K)
    
    return x, R, sigma, K

def EM(y, max_iterations = 100, threshold = 0.05,
        initial_eta = (0.5, 0.5, 0.5, 0.5),
        initial_kalman = (0, 0.1)):
    '''Uses EM Algorithm for estimating A, B, C, D given
        observations of data.
        
        To decode from Elliot(2005):
            - x[k]       == \hat{x}_{k|k}
            - x_N[k]     == \hat{x}_{k|N}
            - R[k]       == \Sigma_{k|k}
            - sigma[k]   == \Sigma_{k+1|k}
            - sigma_N[k] == \Sigma_{k|N}
            - cov_N[k]   == \Sigma_{k-1,k|N}
            - J[k]       == J_k
            - K          == K_N
    '''
    
    #initialize A, B, C, D
    A, B, C, D = initial_eta
    eta = [(A, B, C, D)] #list of parameters at each step

    N = len(y) - 1
    x_N = [None] * (N + 1)      #31
    sigma_N = [None] * (N + 1)  #32
    cov_N = [None] * (N + 1)    #33
    J = [None] * (N + 1)

    #initalize number of iterations and done flag
    i, done = 0, False

    while not done:
        #compute kalman filter with current estimates
        x, R, sigma, K = kalman_filter(y, A, B, C, D, initial_kalman)

        #initialize with results from kalman filter
        x_N[N] = x[N]
        sigma_N[N] = sigma[N]

        #calculating smoothers using backwards recursion
        for k in reversed(range(N)):
            # Equation 34
            J[k] = B * R[k] / sigma[k+1]

            # Equation 35
            x_N[k] = x[k] + J[k] * (x_N[k+1] - (A + B * x[k]))

            # Equation 36
            sigma_N[k] = R[k] + J[k]**2 * (sigma_N[k+1] - sigma[k+1])
        
        #cov uses second index
        #i.e. Sigma_k-1,k|N == cov_N[k]
        # Equation 38
        cov_N[N] = B * (1 - K) * R[N-1]

        for k in reversed(range(1, N)):
            # Equation 37
            val = J[k-1] * R[k]
            val += J[k] * J[k-1] * (cov_N[k+1] - B * R[k])
            cov_N[k] = val

        #helper values
        x_N_2 = [k ** 2 for k in x_N]
        alpha = sum(sigma_N[0:N]) + sum(x_N_2[0:N])
        beta = sum(cov_N[1:]) + sum([a * b for a, b in zip(x[0:N], x[1:])])
        gamma = sum(x_N[1:])
        delta = gamma - x_N[N] + x_N[0]

        #calculating new estimates for A, B, C, D
        A_hat = (alpha * gamma - delta * beta) / (N * alpha - delta ** 2)
        B_hat = (N * beta - gamma * delta) / (N * alpha - delta ** 2)

        C_hat, D_hat = 0, 0
        for k in range(1, N + 1):
            C_hat += sigma_N[k] + x_N[k] ** 2 + A_hat ** 2
            C_hat += B_hat ** 2 * (sigma_N[k-1] + x_N[k-1] ** 2)
            C_hat -= 2 * A_hat * x_N[k]
            C_hat += 2 * A_hat * B_hat * x_N[k-1]
            C_hat -= 2 * B_hat * cov_N[k]
            C_hat -= 2 * B_hat * x_N[k] * x_N[k-1]

        C_hat = C_hat / N
        C_hat = sqrt(C_hat)

        D_hat = sum([a ** 2 for a in y]) - sum([2 * a * b for a, b in zip(y, x_N)])
        D_hat += sum(sigma_N) + sum(x_N_2)
        D_hat = D_hat / (N + 1)
        D_hat = sqrt(D_hat)
        
        #changing initial kalman estimates for the next iteration
        initial_kalman = (x_N[0], sigma_N[0])
        #initial_kalman = (x_N[0], 0.1)
        
        #checking convergence
        dist = (A - A_hat) ** 2 + (B - B_hat) ** 2
        dist += (C - C_hat) ** 2 + (D - D_hat) ** 2
        i += 1
        if i == max_iterations or dist < threshold:
            done = True

        A, B, C, D = A_hat, B_hat, C_hat, D_hat
        eta.append((A, B, C, D))
    return A, B, C, D, eta, x_N

def plot_eta(eta, path = None):
    A_lst, B_lst, C_lst, D_lst = zip(*eta)
    fig, axes = plt.subplots(4, figsize = (8, 15))

    axes[0].plot(A_lst);
    axes[0].set_title('A');
    axes[1].plot(B_lst);
    axes[1].set_title('B');
    axes[2].plot(C_lst);
    axes[2].set_title('C');
    axes[3].plot(D_lst);
    axes[3].set_title('D');

    plt.show()

    fig.savefig(path)

# Half life

In [7]:
class Half_Life(object):
    """
    Half Life test from the Ornstein-Uhlenbeck process 
    """

    def __init__(self):
        self.hl_min = 5.0
        self.hl_max = 30.0
        #self.look_back = 43
        self.half_life = None
        
    
    #Compute half life
    def apply_half_life(self, time_series):
        lag = np.roll(time_series, 1)
        lag[0] = 0
        ret = time_series - lag
        ret[0] = 0

        # adds intercept terms to X variable for regression
        lag2 = sm.add_constant(lag)

        model = sm.OLS(ret, lag2)
        res = model.fit()

        self.half_life = -np.log(2) / res.params[1]
        
    #If half life can be used
    def use(self):
        return (self.half_life < self.hl_max) and (self.half_life > self.hl_min)